In [1]:
from __future__ import annotations

from bisect import bisect
from dataclasses import dataclass, field
import doctest
from functools import reduce
import tqdm

from utilities import extract_ints


In [2]:
@dataclass(frozen=True)
class Range:
  dst: int
  src: int
  length: int

  @classmethod
  def parse_from(cls, l: str) -> 'Range':
    """
    >>> Range.parse_from('50 98 2')
    Range(dst=50, src=98, length=2)
    """
    src, dst, length = [int(c) for c in l.strip().split()]
    return Range(src, dst, length)
  
  def map(self, src: int) -> int:
    """
    >>> r = Range(dst=50, src=98, length=2)
    >>> r.map(98)
    50
    >>> r.map(99)
    51
    >>> r.map(52)
    52
    >>> r.map(8)
    8
    """
    if self.contains(src):
      return self.dst + (src - self.src)
    else:
      return src

  def contains(self, src: int) -> bool:
    return self.src <= src < (self.src + self.length)

  def map_interval(self, interval):
    start = self.src
    end = self.src + self.length
    overlap_start = max(interval.start, start)
    overlap_end = min(interval.stop, end) - 1
    return (
      range(interval.start, min(interval.stop, start)) or None,
      range(self.map(overlap_start), self.map(overlap_end) + 1) if overlap_start < overlap_end else None,
      range(max(interval.start, end), interval.stop) or None
    )

@dataclass
class RangeMap:
  ranges: list[Range]
  srcs: list[int] = field(init=False)

  def __post_init__(self):
    self.ranges.sort(key=lambda r: r.src)
    self.srcs = [r.src for r in self.ranges]

  @classmethod
  def parse_from(cls, txt: str) -> RangeMap:
    """
    >>> RangeMap.parse_from('soil-to-seed map:\\n50 98 2\\n52 50 48')
    RangeMap(ranges=[Range(dst=52, src=50, length=48), Range(dst=50, src=98, length=2)], srcs=[50, 98])
    """
    return RangeMap(ranges=[Range.parse_from(l) for l in txt.strip().splitlines()[1:]])
    
  def map(self, src: int) -> int:
    """
    >>> rm = RangeMap([Range(dst=50, src=98, length=2), Range(dst=52, src=50, length=48)])
    >>> rm.map(79)
    81
    >>> rm.map(14)
    14
    >>> rm.map(55)
    57
    >>> rm.map(50)
    52
    >>> rm = RangeMap([Range(0, 15, 37), Range(37, 52, 2), Range(39, 0, 15)])
    >>> rm.map(14)
    53
    >>> rm.map(81)
    81
    """
    if (i := bisect(self.srcs, src)):
      return self.ranges[i-1].map(src)
    else:
      return src
    
  def map_interval(self, interval):
    start = max(0, bisect(self.srcs, interval.start) - 1)
    end = bisect(self.srcs, interval.stop)
    splits = []
    for r in self.ranges[start:end]:
      left, middle, rest = r.map_interval(interval)
      if left: splits.append(left)
      if middle: splits.append(middle)
      interval = rest
    if interval: splits.append(interval)
    return splits

def parse_puzzle(txt: str):
  sections = txt.split('\n\n')
  seeds = extract_ints(sections[0])
  range_maps = [RangeMap.parse_from(s) for s in sections[1:]]
  return seeds, range_maps


def solution_1(seeds: list[int], range_maps: list[RangeMap]) -> int:
  apply_map = lambda src, rm: rm.map(src)
  apply_maps = lambda maps, src: reduce(apply_map, maps, src)
  return min(apply_maps(range_maps, s) for s in seeds)


def solution_2(seeds: list[int], range_maps: list[RangeMap]) -> int:
  seed_intervals = {range(seeds[i], seeds[i]+seeds[i+1]) for i in range(0, len(seeds), 2)}
  for rm in tqdm.tqdm(range_maps):
    new_intervals = set()
    for s in seed_intervals:
      new_intervals.update(rm.map_interval(s))
    seed_intervals = new_intervals
  return min(s.start for s in seed_intervals)


In [4]:
%%time
# Final answers
with open('../data/day05.txt') as f:
    sections = f.read().split('\n\n')
    seeds = extract_ints(sections[0])
    range_maps = [RangeMap.parse_from(s) for s in sections[1:]]
    print('Part 1: ', solution_1(seeds, range_maps))
    print('Part 2: ', solution_2(seeds, range_maps))


Part 1:  340994526


100%|██████████| 7/7 [00:00<00:00, 2503.64it/s]

Part 2:  52210644
CPU times: user 18 ms, sys: 0 ns, total: 18 ms
Wall time: 18.3 ms
